# <div align="center">8.4 Parameter Initialization Strategies</div>
---------------------------------------------------------------------

you can Find me on Github:
> ###### [ GitHub](https://github.com/lev1khachatryan)

Some optimization algorithms are not iterative by nature and simply solve for a
solution point. Other optimization algorithms are iterative by nature but, when
applied to the right class of optimization problems, converge to acceptable solutions
in an acceptable amount of time regardless of initialization. Deep learning training
algorithms usually do not have either of these luxuries. Training algorithms for deep
learning models are usually iterative in nature and thus require the user to specify
some initial point from which to begin the iterations. Moreover, training deep
models is a sufficiently difficult task that most algorithms are strongly affected by
the choice of initialization. The initial point can determine whether the algorithm
converges at all, with some initial points being so unstable that the algorithm
encounters numerical difficulties and fails altogether. When learning does converge,
the initial point can determine how quickly learning converges and whether it
converges to a point with high or low cost. Also, points of comparable cost
can have wildly varying generalization error, and the initial point can affect the
generalization as well.

Modern initialization strategies are simple and heuristic. Designing improved
initialization strategies is a difficult task because neural network optimization is
not yet well understood. Most initialization strategies are based on achieving some
nice properties when the network is initialized. However, we do not have a good
understanding of which of these properties are preserved under which circumstances
after learning begins to proceed. A further difficulty is that some initial points
may be beneficial from the viewpoint of optimization but detrimental from the
viewpoint of generalization. Our understanding of how the initial point affects
generalization is especially primitive, offering little to no guidance for how to select
the initial point.

Perhaps the only property known with complete certainty is that the initial
parameters need to “break symmetry” between different units. If two hidden
units with the same activation function are connected to the same inputs, then
these units must have different initial parameters. If they have the same initial
parameters, then a deterministic learning algorithm applied to a deterministic cost
and model will constantly update both of these units in the same way. Even if the
model or training algorithm is capable of using stochasticity to compute different
updates for different units (for example, if one trains with dropout), it is usually
best to initialize each unit to compute a different function from all of the other
units. This may help to make sure that no input patterns are lost in the null
space of forward propagation and no gradient patterns are lost in the null space
of back-propagation. The goal of having each unit compute a different function motivates random initialization of the parameters. We could explicitly search
for a large set of basis functions that are all mutually different from each other,
but this often incurs a noticeable computational cost. For example, if we have at
most as many outputs as inputs, we could use Gram-Schmidt orthogonalization
on an initial weight matrix, and be guaranteed that each unit computes a very
different function from each other unit. Random initialization from a high-entropy
distribution over a high-dimensional space is computationally cheaper and unlikely
to assign any units to compute the same function as each other.

Typically, we set the biases for each unit to heuristically chosen constants, and
initialize only the weights randomly. Extra parameters, for example, parameters
encoding the conditional variance of a prediction, are usually set to heuristically
chosen constants much like the biases are.

We almost always initialize all the weights in the model to values drawn
randomly from a Gaussian or uniform distribution. The choice of Gaussian
or uniform distribution does not seem to matter very much, but has not been
exhaustively studied. The scale of the initial distribution, however, does have a
large effect on both the outcome of the optimization procedure and on the ability
of the network to generalize.

Larger initial weights will yield a stronger symmetry breaking effect, helping
to avoid redundant units. They also help to avoid losing signal during forward or
back-propagation through the linear component of each layer—larger values in the
matrix result in larger outputs of matrix multiplication. Initial weights that are
too large may, however, result in exploding values during forward propagation or
back-propagation. In recurrent networks, large weights can also result in chaos
(such extreme sensitivity to small perturbations of the input that the behavior
of the deterministic forward propagation procedure appears random). To some
extent, the exploding gradient problem can be mitigated by gradient clipping
(thresholding the values of the gradients before performing a gradient descent step).
Large weights may also result in extreme values that cause the activation function
to saturate, causing complete loss of gradient through saturated units. These
competing factors determine the ideal initial scale of the weights.

The perspectives of regularization and optimization can give very different
insights into how we should initialize a network. The optimization perspective
suggests that the weights should be large enough to propagate information successfully, but some regularization concerns encourage making them smaller. The use
of an optimization algorithm such as stochastic gradient descent that makes small
incremental changes to the weights and tends to halt in areas that are nearer to
the initial parameters (whether due to getting stuck in a region of low gradient, or due to triggering some early stopping criterion based on overfitting) expresses a
prior that the final parameters should be close to the initial parameters. Recall
from section 7.8 that gradient descent with early stopping is equivalent to weight
decay for some models. In the general case, gradient descent with early stopping is
not the same as weight decay, but does provide a loose analogy for thinking about
the effect of initialization. We can think of initializing the parameters θ to θ0 as
being similar to imposing a Gaussian prior p(θ) with mean θ0 . From this point
of view, it makes sense to choose θ0 to be near 0. This prior says that it is more
likely that units do not interact with each other than that they do interact. Units
interact only if the likelihood term of the objective function expresses a strong
preference for them to interact. On the other hand, if we initialize θ0 to large
values, then our prior specifies which units should interact with each other, and
how they should interact.

Some heuristics are available for choosing the initial scale of the weights. One
heuristic is to initialize the weights of a fully connected layer with m inputs and
n outputs by sampling each weight from U (− 1/√m , 1/√m), while Glorot and Bengio
(2010) suggest using the ***normalized initialization***

<img src='asset/8_4/1.png'>

This latter heuristic is designed to compromise between the goal of initializing
all layers to have the same activation variance and the goal of initializing all
layers to have the same gradient variance. ***The formula is derived using the
assumption that the network consists only of a chain of matrix multiplications,
with no nonlinearities.*** Real neural networks obviously violate this assumption,
but many strategies designed for the linear model perform reasonably well on its
nonlinear counterparts

Saxe et al. (2013) recommend initializing to random orthogonal matrices, with
a carefully chosen scaling or gain factor g that accounts for the nonlinearity applied
at each layer. They derive specific values of the scaling factor for different types of
nonlinear activation functions. This initialization scheme is also motivated by a
model of a deep network as a sequence of matrix multiplies without nonlinearities.
Under such a model, this initialization scheme guarantees that the total number of
training iterations required to reach convergence is independent of depth.

Increasing the scaling factor g pushes the network toward the regime where
activations increase in norm as they propagate forward through the network and
gradients increase in norm as they propagate backward. Sussillo (2014) showed
that setting the gain factor correctly is sufficient to train networks as deep as 1,000 layers, without needing to use orthogonal initializations. A key insight of
this approach is that in feedforward networks, activations and gradients can grow
or shrink on each step of forward or back-propagation, following a random walk
behavior. This is because feedforward networks use a different weight matrix at
each layer. If this random walk is tuned to preserve norms, then feedforward
networks can mostly avoid the vanishing and exploding gradients problem that
arises when the same weight matrix is used at each step, described in section 8.2.5.

Unfortunately, these optimal criteria for initial weights often do not lead to
optimal performance. This may be for three different reasons. First, we may
be using the wrong criteria—it may not actually be beneficial to preserve the
norm of a signal throughout the entire network. Second, the properties imposed
at initialization may not persist after learning has begun to proceed. Third, the
criteria might succeed at improving the speed of optimization but inadvertently
increase generalization error. In practice, we usually need to treat the scale of the
weights as a hyperparameter whose optimal value lies somewhere roughly near but
not exactly equal to the theoretical predictions.

One drawback to scaling rules that set all of the initial weights to have the
same standard deviation, such as 1/√m, is that every individual weight becomes
extremely small when the layers become large. Martens (2010) introduced an
alternative initialization scheme called ***sparse initialization*** in which each unit is
initialized to have exactly k non-zero weights. The idea is to keep the total amount
of input to the unit independent from the number of inputs m without making the
magnitude of individual weight elements shrink with m. Sparse initialization helps
to achieve more diversity among the units at initialization time. However, it also
imposes a very strong prior on the weights that are chosen to have large Gaussian
values. Because it takes a long time for gradient descent to shrink “incorrect” large
values, this initialization scheme can cause problems for units such as maxout units
that have several filters that must be carefully coordinated with each other

When computational resources allow it, it is usually a good idea to treat the
initial scale of the weights for each layer as a hyperparameter, and to choose these
scales using a hyperparameter search algorithm described in section 11.4.2, such
as random search. The choice of whether to use dense or sparse initialization
can also be made a hyperparameter. Alternately, one can manually search for
the best initial scales. A good rule of thumb for choosing the initial scales is to
look at the range or standard deviation of activations or gradients on a single
minibatch of data. If the weights are too small, the range of activations across the
minibatch will shrink as the activations propagate forward through the network.
By repeatedly identifying the first layer with unacceptably small activations and increasing its weights, it is possible to eventually obtain a network with reasonable
initial activations throughout. If learning is still too slow at this point, it can be
useful to look at the range or standard deviation of the gradients as well as the
activations. This procedure can in principle be automated and is generally less
computationally costly than hyperparameter optimization based on validation set
error because it is based on feedback from the behavior of the initial model on a
single batch of data, rather than on feedback from a trained model on the validation
set. While long used heuristically, this protocol has recently been specified more
formally and studied by Mishkin and Matas (2015).

So far we have focused on the initialization of the weights. Fortunately,
initialization of other parameters is typically easier

The approach for setting the biases must be coordinated with the approach
for settings the weights. Setting the biases to zero is compatible with most weight
initialization schemes. There are a few situations where we may set some biases to
non-zero values:

### 1

If a bias is for an output unit, then it is often beneficial to initialize the bias to
obtain the right marginal statistics of the output. To do this, we assume that
the initial weights are small enough that the output of the unit is determined
only by the bias. This justifies setting the bias to the inverse of the activation
function applied to the marginal statistics of the output in the training set.
For example, if the output is a distribution over classes and this distribution
is a highly skewed distribution with the marginal probability of class i given
by element ci of some vector c, then we can set the bias vector b by solving
the equation softmax(b) = c. This applies not only to classifiers but also to
models we will encounter in Part III, such as autoencoders and Boltzmann
machines. These models have layers whose output should resemble the input
data x, and it can be very helpful to initialize the biases of such layers to
match the marginal distribution over x

### 2

Sometimes we may want to choose the bias to avoid causing too much
saturation at initialization. For example, we may set the bias of a ReLU
hidden unit to 0.1 rather than 0 to avoid saturating the ReLU at initialization.
This approach is not compatible with weight initialization schemes that do
not expect strong input from the biases though. For example, it is not
recommended for use with random walk initialization (Sussillo, 2014).

Another common type of parameter is a variance or precision parameter. For
example, we can perform linear regression with a conditional variance estimate
using the model

<img src='asset/8_4/2.png'>

where β is a precision parameter. We can usually initialize variance or precision
parameters to 1 safely. Another approach is to assume the initial weights are close
enough to zero that the biases may be set while ignoring the effect of the weights,
then set the biases to produce the correct marginal mean of the output, and set
the variance parameters to the marginal variance of the output in the training set.

Besides these simple constant or random methods of initializing model parameters, it is possible to initialize model parameters using machine learning. A common
strategy is to initialize a supervised model with
the parameters learned by an unsupervised model trained on the same inputs.
One can also perform supervised training on a related task. Even performing
supervised training on an unrelated task can sometimes yield an initialization that
offers faster convergence than a random initialization. Some of these initialization
strategies may yield faster convergence and better generalization because they
encode information about the distribution in the initial parameters of the model.
Others apparently perform well primarily because they set the parameters to have
the right scale or set different units to compute different functions from each other.